In [3]:
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
from numpy.polynomial.polynomial import polyfit
from scipy import signal
from scipy.stats import norm

import matplotlib

import matplotlib.pyplot as plt
import netCDF4 as nc
from matplotlib import pyplot
from matplotlib import colors as mcolors
import matplotlib.gridspec as gridspec
from scipy.stats import norm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import gcm_filters

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter


In [2]:
def compute_spectrum(KE_spectre):
    dx=1/(dx_min/1000)
    print(dx_min)
    no=0
    ff_KE_spectre=np.zeros(KE_spectre.shape)
    ff_KE_spectreki0=np.zeros(KE_spectre.shape)
    # len(KE_spectre_20[:,0])
    for j in range(len(KE_spectre[0,:])):
        ff_KE_spectre[:,j], ff_KE_spectreki0[:,j]= signal.welch(KE_spectre[:,j], fs=dx,nperseg=int(len(KE_spectre[:,0])), window='hanning', noverlap=no,nfft=2*int(len(KE_spectre[:,0])-1),  detrend='linear', return_onesided=True, scaling='spectrum')

    mean_f0_KE_spectre = np.nanmean(ff_KE_spectre,axis=1)
    mean_fi0_KE_spectre = np.nanmean(ff_KE_spectreki0,axis=1)
    return mean_f0_KE_spectre, mean_fi0_KE_spectre

In [6]:
Coordfile = xr.open_dataset('/data/vdi/tbrivoal/PRE_PROCESSING_IMMERSE/BATHY/DOMAINcfgs/domain_cfg_init.nc', drop_variables={"x", "y",})
DSTART = '2017-01-01'
DEND = '2018-01-01'


Ufile_AGRIF_36 = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/EXP02_AGRIF_finaldomain_bathycorrected_qco_boost2_noslip/eNEATL36_1h_KE_filt36_daymean.nc', chunks={'time_counter': 50}).sel(time_counter=slice(DSTART, DEND))

Ufile_AGRIF_20 = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/EXP02_AGRIF_finaldomain_bathycorrected_qco_boost2_noslip/eNEATL36_1h_KE_filt20_daymean.nc', chunks={'time_counter': 50}).sel(time_counter=slice(DSTART, DEND))

Ufile_AGRIF_8 = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/EXP02_AGRIF_finaldomain_bathycorrected_qco_boost2_noslip/eNEATL36_1h_KE_filt8_daymean.nc', chunks={'time_counter': 50}).sel(time_counter=slice(DSTART, DEND))


###################

Ufile_TWIN_36 = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/eNEATL36_trunk_r4_2-RC_r15113_IFS_EXP02_2017_2018_AGRIFTWIN_TRUE/eNEATL36_1h_KE_filt36_daymean.nc', chunks={'time_counter': 50}).sel(time_counter=slice(DSTART, DEND))

Ufile_TWIN_20 = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/eNEATL36_trunk_r4_2-RC_r15113_IFS_EXP02_2017_2018_AGRIFTWIN_TRUE/eNEATL36_1h_KE_filt20_daymean.nc', chunks={'time_counter': 50}).sel(time_counter=slice(DSTART, DEND))

Ufile_TWIN_8 = xr.open_dataset('/data/vdi/tbrivoal/RUNS_DATA/eNEATL36_trunk_r4_2-RC_r15113_IFS_EXP02_2017_2018_AGRIFTWIN_TRUE/eNEATL36_1h_KE_filt8_daymean.nc', chunks={'time_counter': 50}).sel(time_counter=slice(DSTART, DEND))


In [7]:
dxe = Coordfile.e1t.squeeze()
dye = Coordfile.e2t.squeeze()
dxn = Coordfile.e1v.squeeze()
dyn = Coordfile.e2v.squeeze()
dx_min = 3000 #min(dxe.min(),dye.min(),dxn.min(),dyn.min())
lon=Coordfile.glamt.squeeze()
lat=Coordfile.gphit.squeeze()
bathy=Coordfile.bathy_metry.squeeze()
